``` 
We used colab for trainng model
```

change class number in labels

In [ ]:
import os

def change_class_number(label_dir):
   # 디렉토리 내의 모든 파일을 순회
  for filename in os.listdir(label_dir):
      if filename.endswith(".txt"):  # 텍스트 파일인지 확인
          file_path = os.path.join(label_dir, filename)
          with open(file_path, 'r') as file:
              lines = file.readlines()

          # 클래스 번호 변경
          with open(file_path, 'w') as file:
              for line in lines:
                  parts = line.strip().split()
                  parts[0] = str(int(parts[0])+91)  # 새 클래스 번호로 변경 : coco2014 데이터셋 90개, 1~90
                  file.write(" ".join(parts) + '\n')

change_class_number('/content/firehazards-7/test/labels'); # test set
change_class_number('/content/firehazards-7/train/labels'); # train set
change_class_number('/content/firehazards-7/valid/labels'); # valid set

Extract labels text from annotation.json

In [ ]:
import json
from pycocotools.coco import COCO
import os

def convert_coco_json_to_yolo(coco_json_path, output_dir):
    # COCO 데이터셋 로드
    coco = COCO(coco_json_path)

    # 모든 이미지에 대한 정보를 가져옵니다
    img_ids = coco.getImgIds()
    img_info = coco.loadImgs(img_ids)

    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img in img_info:
        # 각 이미지에 대한 주석 정보를 가져옵니다
        ann_ids = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
        anns = coco.loadAnns(ann_ids)

        # 출력 파일 준비
        output_filename = os.path.join(output_dir, f"{img['file_name'].replace('.jpg', '.txt')}")
        with open(output_filename, 'w') as f:
            for ann in anns:
                # 클래스 ID
                class_id = ann['category_id']
                # 바운딩 박스 정보 추출 및 변환
                x_center, y_center, width, height = ann['bbox']
                x_center += width / 2
                y_center += height / 2
                # 이미지 크기로 나누어 상대적 위치 계산
                x_center /= img['width']
                y_center /= img['height']
                width /= img['width']
                height /= img['height']
                # YOLO 형식으로 파일에 쓰기
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# train label
coco_json_path = '/content/annotations/instances_train2014.json' # JSON 파일 위치
output_dir = '/content/annotations/train_label' # YOLO 라벨 파일을 저장할 위치
# run
convert_coco_json_to_yolo(coco_json_path, output_dir)

# valid label
coco_json_path = '/content/annotations/instances_val2014.json' # JSON 파일 위치
output_dir = '/content/annotations/valid_label' # YOLO 라벨 파일을 저장할 위치

# run
convert_coco_json_to_yolo(coco_json_path, output_dir)

Copy and merge images and labels

In [ ]:
!find /content/train2014/ -type f -print0 | xargs -0 -I {} cp {} firehazards-7/train/images/
!find /content/val2014/ -type f -print0 | xargs -0 -I {} cp {} firehazards-7/valid/images/
!find /content/annotations/valid_label/ -type f -print0 | xargs -0 -I {} cp {} firehazards-7/valid/labels/
!find /content/annotations/train_label/ -type f -print0 | xargs -0 -I {} cp {} firehazards-7/train/labels/

Check all labels are within range

In [ ]:
import os
import glob

# 라벨 체크를 위한 코드
def check_labels(label_dir, class_range):
    """
    Check if all labels in the given directory are within the specified class range.

    Parameters:
    - label_dir: Directory containing label files.
    - class_range: Tuple of (min_class, max_class) indicating the range of valid class indices.

    Returns:
    - A list of files that have labels outside the specified class range.
    """
    out_of_range_files = []
    for label_file in glob.glob(os.path.join(label_dir, "*.txt")):
        with open(label_file, 'r') as file:
            lines = file.readlines()
            for line in lines:
                class_id = int(line.split()[0])
                if class_id < class_range[0] or class_id > class_range[1]:
                    out_of_range_files.append(label_file)
                    break  # No need to check more labels in this file
    return out_of_range_files

# 예제 사용법
label_directory = "/content/annotations/train_label"  # 라벨 파일이 위치한 디렉토리.
min_class_id = 1
max_class_id = 90  # 클래스 인덱스가 1에서 90까지 (coco 2014, 총 90개 클래스)
out_of_range = check_labels(label_directory, (min_class_id, max_class_id))

print("범위를 벗어난 라벨이 있는 파일들:", out_of_range)